**Generating Baby Names**

In [21]:
import sys
import re
import requests
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM
from tensorflow.keras.models import load_model

In [22]:
r = requests.get('https://cs.stanford.edu/people/karpathy/namesGenUnique.txt')
raw_txt = r.text
print(raw_txt[:100])

jka
Dillie
Ryine
Cherita
Dasher
Chailine
Frennide
Gremaley
Patj
Handi
Gully
Wennie
Ferentra
Jixandli


In [23]:
len(raw_txt)

52127

**Processing Text**

In [24]:
raw_txt = raw_txt.replace('\n' , ' ')

In [25]:
raw_txt = re.sub('[-.0-9:]' , '' , raw_txt)

In [26]:
raw_txt1 = raw_txt.lower()

In [27]:
set(raw_txt1)

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [28]:
len1 = len(set(raw_txt1))
print (len1)

27


In [29]:
chars = sorted(list(set(raw_txt1)))

In [30]:
arr = np.arange(0, len1)

In [31]:
char_to_ix = {}
ix_to_char = {}
for i in range(len1):
  char_to_ix[chars[i]] = arr[i]
  ix_to_char[arr[i]] = chars[i]

In [32]:
maxlen = 5
x_data = []
y_data = []
for i in range(0, len(raw_txt1) - maxlen, 1):
  in_seq = raw_txt1[i: i + maxlen]
  out_seq = raw_txt1[i + maxlen]
  x_data.append([char_to_ix[char] for char in in_seq])
  y_data.append([char_to_ix[out_seq]])
nb_chars = len(x_data)
print('Text corpus: {}'.format(nb_chars))
print('Sequences # ', int(len(x_data) / maxlen))

Text corpus: 52038
Sequences #  10407


In [33]:
x = np.reshape(x_data, (nb_chars , maxlen , 1))
x = x/float(len(chars))
x.shape

(52038, 5, 1)

We convert the target sequence into categorical columns

In [34]:
y = tf.keras.utils.to_categorical(y_data)
y.shape

(52038, 27)

**Defining Model**

In [35]:
model = tf.keras.Sequential(
    [
     tf.keras.layers.LSTM(256,input_shape = (maxlen, 1),return_sequences = True),
     tf.keras.layers.LSTM(256,return_sequences = True),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.LSTM(64),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(len(y[1]),activation='softmax')
    ]
)

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 5, 256)            264192    
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 256)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 27)                1755      
Total params: 873,435
Trainable params: 873,435
Non-trainable params: 0
________________________________________________

In [37]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

**Creating Checkpoints**

In [38]:
filepath = "model_weights_babynames.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
model_callbacks = [checkpoint]

**Training**

In [39]:
model.fit(x, y, epochs = 100, batch_size = 32 , callbacks = model_callbacks)

Epoch 1/100
1627/1627 [==============================] - 98s 57ms/step - loss: 2.7783

Epoch 00001: loss improved from inf to 2.77826, saving model to model_weights_babynames.hdf5
Epoch 2/100
1627/1627 [==============================] - 95s 58ms/step - loss: 2.5695

Epoch 00002: loss improved from 2.77826 to 2.56952, saving model to model_weights_babynames.hdf5
Epoch 3/100
1627/1627 [==============================] - 95s 58ms/step - loss: 2.5041

Epoch 00003: loss improved from 2.56952 to 2.50411, saving model to model_weights_babynames.hdf5
Epoch 4/100
1627/1627 [==============================] - 95s 59ms/step - loss: 2.4685

Epoch 00004: loss improved from 2.50411 to 2.46852, saving model to model_weights_babynames.hdf5
Epoch 5/100
1627/1627 [==============================] - 96s 59ms/step - loss: 2.4372

Epoch 00005: loss improved from 2.46852 to 2.43723, saving model to model_weights_babynames.hdf5
Epoch 6/100
1627/1627 [==============================] - 96s 59ms/step - loss: 2.417

KeyboardInterrupt: ignored

**Prediction**

In [40]:
pattern = []
seed = 'handi'
for i in seed:
  value = char_to_ix[i]
  pattern.append(value)

In [41]:
print(seed)
n_vocab = len(chars)

handi


In [42]:
print(seed)
n_vocab = len(chars)
for i in range(100):
  X = np.reshape(pattern , (1, len(pattern) , 1))
  X = X/float(n_vocab)
  int_prediction = model.predict(X , verbose = 0)
  index = np.argmax(int_prediction)
  prediction = ix_to_char[index]
  print(prediction)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

handi
e
 
s
h
e
r
r
a
 
m
a
r
t
i
n
a
 
a
l
e
g
e
 
s
h
e
r
r
a
 
m
a
r
t
i
n
a
 
a
l
e
g
e
 
s
h
e
r
r
a
 
m
a
r
t
i
n
a
 
a
l
e
g
e
 
s
h
e
r
r
a
 
m
a
r
t
i
n
a
 
a
l
e
g
e
 
s
h
e
r
r
a
 
m
a
r
t
i
n
a


**Saving/Reusing Model**

In [43]:
model.save('baby_names_model.h5')

In [44]:
saved_model = load_model('baby_names_model.h5')